In [2]:
#import sys
#!{sys.executable} -m pip install torch gym_conservation "ray[rllib, tune]"

In [1]:
import gym
import gym_climate
import gym_conservation
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

/home/cboettig/.local/share/virtualenvs/rl-ray-demo-y9hdSGSh/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
## Training
#env = make_vec_env("ays-v0", n_envs=4)
#model = A2C("MlpPolicy", env, verbose=1,tensorboard_log="/var/log/tensorboard/carl")
#model.learn(total_timesteps=10)
#model.save("ays-v0-A2C")

# Evaluation
model = A2C.load("ays-v0-A2C")
eval_env = Monitor(gym.make("ays-v0"))
score = evaluate_policy(model, eval_env, n_eval_episodes=10)
score


(21.0, 0.0)

In [7]:

import argparse
import gym
import numpy as np
import os

import ray
from ray import tune
from ray.rllib import agents
import torch

import gym_conservation
import gym_fishing

## rllib ignores gym registered names, need to register manually:
## note these envs were not written to take a single parameter dictionary ("config")
tune.register_env("conservation-v6", lambda config: gym_conservation.envs.NonStationaryV6())
tune.register_env("conservation-v5", lambda config: gym_conservation.envs.NonStationaryV5())
tune.register_env("fishing-v0", lambda config: gym_fishing.envs.FishingEnv())
tune.register_env("fishing-v1", lambda config: gym_fishing.envs.FishingCtsEnv())

os.environ["RLLIB_NUM_GPUS"] = str(torch.cuda.device_count())
os.environ.get("RLLIB_NUM_GPUS")


os.environ['RAY_OBJECT_STORE_ALLOW_SLOW_STORAGE'] = '1'

In [66]:
# Read in an ExperimentAnalysis from a json state file
from ray.rllib.utils.test_utils import check_learning_achieved
#results_path = "~/ray_results/ARS/"
results_path = "~/ray_results/TD3/"

analysis = tune.ExperimentAnalysis(experiment_checkpoint_path=results_path)

# status = check_learning_achieved(analysis, 150) # TD3 doesn't meet goal

2022-02-17 22:38:28,262	INFO experiment_analysis.py:695 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.


ok


In [65]:

config = analysis.get_best_trial(metric="episode_reward_mean", mode="max").last_result["config"]


#analysis.dataframe(metric="episode_reward_mean", mode="max")
analysis.get_best_trial(metric="episode_reward_mean", mode="max").last_result



{'episode_reward_max': 73.61594719906185,
 'episode_reward_min': 32.51658480751109,
 'episode_reward_mean': 44.41801731162805,
 'episode_len_mean': 497.92,
 'episode_media': {},
 'episodes_this_iter': 2,
 'policy_reward_min': {},
 'policy_reward_max': {},
 'policy_reward_mean': {},
 'custom_metrics': {},
 'hist_stats': {'episode_reward': [51.71604481725184,
   46.21605940677299,
   54.88445713546756,
   41.117737699487066,
   53.04154396479634,
   40.296141160220074,
   39.71060423141501,
   38.6883339421757,
   37.337722913348024,
   39.869549477468325,
   42.45428257420593,
   44.300189259634166,
   59.18940100120667,
   47.355386740495376,
   38.578367892571954,
   49.55682233591693,
   42.15501828401235,
   58.9203660323364,
   35.222093950334205,
   36.80933707249204,
   60.05479419877589,
   44.36395543100008,
   35.20693692672356,
   40.3374343870914,
   43.23732838303333,
   42.1611176160673,
   40.54241242572631,
   41.25146119890267,
   45.2751185797469,
   39.259403762923505

In [57]:
# extract the hyper-parameter config
config = analysis.get_best_trial(metric="episode_reward_mean", mode="max").last_result["config"]
config.pop("in_evaluation", None)


In [60]:
## Okay, now we can initialize a model 
model = agents.ddpg.TD3Trainer(config)


2022-02-17 19:54:26,934	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-02-17 19:54:26,936	WARNING deprecation.py:46 -- DeprecationWarning: `config['buffer_size']` has been deprecated. Use `config['replay_buffer_config']['capacity']` instead. This will raise an error in the future!


In [61]:
model.restore("/home/cboettig/ray_results/TD3/")


FileNotFoundError: [Errno 2] No such file or directory: '/home/cboettig/ray_results/TD3/.tune_metadata'

In [ ]:
import json
params = "/home/cboettig/ray_results/TD3/TD3_conservation-v6_094ac_00000_0_2022-02-16_15-35-14/params.json"
with open(params) as json_file:
    data = json.load(json_file)
data

In [ ]:

# Evaluate the trained Trainer (and render each timestep to the shell's output).
trainer.evaluate()



{'evaluation': {'episode_reward_max': 1.2274416977413574,
  'episode_reward_min': 0.75,
  'episode_reward_mean': 0.8675837622445943,
  'episode_len_mean': 3.8,
  'episode_media': {},
  'episodes_this_iter': 10,
  'policy_reward_min': {},
  'policy_reward_max': {},
  'policy_reward_mean': {},
  'custom_metrics': {},
  'hist_stats': {'episode_reward': [0.80625,
    0.80625,
    1.2274416977413574,
    0.75,
    0.75,
    1.0980312748711576,
    0.80625,
    0.80625,
    0.8191146498334291,
    0.80625],
   'episode_lengths': [2, 2, 9, 1, 1, 14, 2, 2, 3, 2]},
  'sampler_perf': {'mean_raw_obs_processing_ms': 0.3598704934120178,
   'mean_inference_ms': 1.170910894870758,
   'mean_action_processing_ms': 0.11758009592692058,
   'mean_env_wait_ms': 0.10723620653152466,
   'mean_env_render_ms': 0.0},
  'off_policy_estimator': {},
  'timesteps_this_iter': 0}}